<a href="https://colab.research.google.com/github/mnaaseri/ner_search/blob/main/ner_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/search/semantic-search/ner-search/ner-powered-search.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/search/semantic-search/ner-search/ner-powered-search.ipynb)

# NER Semantic Search Using Pinecone Indexing

# Install Dependencies

In [ ]:
!pip install sentence_transformers pinecone-client datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00


## Import Libraries

# Load and Prepare Dataset

You can find the dataset I used [here](https://www.kaggle.com/datasets/mehranrafiee/persian-articles-for-labeling)

In [ ]:
import pandas as pd
df = pd.read_csv("/virgool_dataset.csv")
df

,sentence,subject,link,label
0,\nعصر امروز، عصر اطّلاعات بوده و کامپیوتر به ش...,راهنمای جامع برنامه‌‌نویس شدن,https://virgool.io/@pouyan_01001010/ultimate-p...,['برنامه\u200cنویسی' 'کامپیوتر' 'نرم\u200cافزا...
1,\nبیش از دو ماه از قطعی اینترنت بین‌الملل در ۲...,گزارشی از طولانی‌ترین قطعی اینترنت در ایران,https://virgool.io/@ali/%DA%AF%D8%B2%D8%A7%D8%...,['اینترنت']
2,\nخیر، من استاد دانشگاه یا معلم مدرسه نیستم. ا...,آسیب‌هایی که تقلب کردن برای من داشت,https://virgool.io/@ARKhoshghalb/%DA%86%D8%B1%...,['تقلب' 'مدرسه' 'دانشگاه' 'تحصیل']
3,\nمن دو سال دیگر کنکور رشته انسانی دارم، حدود ...,کار، کار کنکور است,https://virgool.io/@HesamShams1/%D8%A8%D8%A7%D...,['کنکور' 'کنکور لعنتی' 'علوم انسانی' 'دانشگاه']
4,\nنویسندگی توی دنیای امروز کار آنچنان سختی نیس...,شروع نویسندگی,https://virgool.io/@ar.exhu/%D8%B4%D8%B1%D9%88...,['نویسندگی' 'بلاگینگ' 'شبکه اجتماعی' 'ویرگول' ...
...,...,...,...,...
11863,\nدر این مقاله می خواهیم به ۱۰ نکته مهم برای ا...,۱۰ نکته مهم برای انتخاب کلمات کلیدی مناسب,https://virgool.io/@clubseokaran/%DB%B1%DB%B0-...,['کلمات کلیدی' 'سئو سایت' 'بهینه سازی سایت' 'ب...
11864,\n👈🏻ماهی یکی دوتا فیلم \n👈🏻اکسل و گمز 💪🏻 ياد م...,۱۳۹۹ برای من,https://virgool.io/@eliiya/%DB%B1%DB%B3%DB%B9%...,[]
11865,\nبه مناسبت قرنطینه چالش شخصی با عنوان «چالش پ...,۱۵ سریال برتر در ژانر درام جنایی,https://virgool.io/@javad.oftadeh/%DB%B1%DB%B5...,['سریال' 'بریکینگ بد' 'دارک' 'معرفی سریال' 'پی...
11866,\n\n۱- خطوط روی زبان هر فردی منحصر به خود اوست...,۲۰ حقیقت جالب راجع به بدن خود که شاید نمی دانستید,NaN,[]


In [ ]:
df = df.dropna()
df.head()

,sentence,subject,link,label
0,\nعصر امروز، عصر اطّلاعات بوده و کامپیوتر به ش...,راهنمای جامع برنامه‌‌نویس شدن,https://virgool.io/@pouyan_01001010/ultimate-p...,['برنامه\u200cنویسی' 'کامپیوتر' 'نرم\u200cافزا...
1,\nبیش از دو ماه از قطعی اینترنت بین‌الملل در ۲...,گزارشی از طولانی‌ترین قطعی اینترنت در ایران,https://virgool.io/@ali/%DA%AF%D8%B2%D8%A7%D8%...,['اینترنت']
2,\nخیر، من استاد دانشگاه یا معلم مدرسه نیستم. ا...,آسیب‌هایی که تقلب کردن برای من داشت,https://virgool.io/@ARKhoshghalb/%DA%86%D8%B1%...,['تقلب' 'مدرسه' 'دانشگاه' 'تحصیل']
3,\nمن دو سال دیگر کنکور رشته انسانی دارم، حدود ...,کار، کار کنکور است,https://virgool.io/@HesamShams1/%D8%A8%D8%A7%D...,['کنکور' 'کنکور لعنتی' 'علوم انسانی' 'دانشگاه']
4,\nنویسندگی توی دنیای امروز کار آنچنان سختی نیس...,شروع نویسندگی,https://virgool.io/@ar.exhu/%D8%B4%D8%B1%D9%88...,['نویسندگی' 'بلاگینگ' 'شبکه اجتماعی' 'ویرگول' ...


In [ ]:
# select first 1000 characters
df["sentence"] = df["sentence"].str[:500]
# join article title and the text
df["title_text"] = df["subject"] + ". " + df["sentence"]

<ipython-input-18-6c8d09b63a74>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["sentence"] = df["sentence"].str[:500]
<ipython-input-18-6c8d09b63a74>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["title_text"] = df["subject"] + ". " + df["sentence"]


# Initialize NER Model

In [ ]:
import torch

# set device to GPU if available
device = torch.cuda.current_device() if torch.cuda.is_available() else None

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

model_id = "HooshvareLab/bert-base-parsbert-ner-uncased"

# load the tokenizer from huggingface
tokenizer = AutoTokenizer.from_pretrained(
    model_id
)
# load the NER model from huggingface
model = AutoModelForTokenClassification.from_pretrained(
    model_id
)
# load the tokenizer and model into a NER pipeline
nlp = pipeline(
    "ner",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="max",
    device=device
)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = "تهران پایتخت ایران است"
# use the NER pipeline to extract named entities from the text
nlp(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'location',
  'score': 0.9996954,
  'word': 'تهران',
  'start': 0,
  'end': 5},
 {'entity_group': 'location',
  'score': 0.9996921,
  'word': 'ایران',
  'start': 13,
  'end': 18}]

Our NER pipeline is working as expected and accurately extracting entities from the text.

# Initialize Retriever

In [ ]:
from sentence_transformers import SentenceTransformer

# load the model from huggingface
retriever = SentenceTransformer(
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2',
    device=device
)
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# Initialize Pinecone Index

 For this, we need a free [API key](https://app.pinecone.io/); you can find your environment in the [Pinecone console](https://app.pinecone.io) under **API Keys**.

In [ ]:
import os
import pinecone

api_key = os.environ.get("PINECONE_API_KEY") or "YOUR-API-KEY"
env = os.environ.get("PINECONE_ENVIRONMENT") or "YOUR-ENVIRONMENT"
pinecone.init(api_key=api_key, enviroment=env)

ERROR:root:init had unexpected keyword argument(s): enviroment


Now we can create our vector index. We will name it `ner-search` (feel free to chose any name you prefer). We specify the metric type as `cosine` and dimension as `768` as these are the vector space and dimensionality of the vectors output by the retriever model.

In [ ]:
index_name = "ner-search"

In [ ]:
# check if the ner-search index exists
if index_name not in pinecone.list_indexes():
    # create the index if it does not exist
    pinecone.create_index(
        index_name,
        dimension=768,
        metric="cosine"
    )

# connect to ner-search index we created
index = pinecone.Index(index_name)

# Generate Embeddings and Upsert

In [ ]:
def extract_named_entities(text_batch):
    # extract named entities using the NER pipeline
    extracted_batch = nlp(text_batch)
    entities = []
    # loop through the results and only select the entity names
    for text in extracted_batch:
        ne = [entity["word"] for entity in text]
        entities.append(ne)
    return entities

In [ ]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

# we will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # extract batch
    batch = df.iloc[i:i_end].copy()
    # generate embeddings for batch
    emb = retriever.encode(batch["title_text"].tolist()).tolist()
    # extract named entities from the batch
    entities = extract_named_entities(batch["title_text"].tolist())
    # remove duplicate entities from each record
    batch["named_entities"] = [list(set(entity)) for entity in entities]
    batch = batch.drop('title_text', axis=1)
    # get metadata
    meta = batch.to_dict(orient="records")
    # create unique IDs
    ids = [f"{idx}" for idx in range(i, i_end)]
    # add all to upsert list
    to_upsert = list(zip(ids, emb, meta))
    # upsert/insert these records to pinecone
    _ = index.upsert(vectors=to_upsert)

# check that we have all vectors in index
index.describe_index_stats()

  0%|          | 0/186 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'dimension': 768,
 'index_fullness': 0.1,
 'namespaces': {'': {'vector_count': 20000}},
 'total_vector_count': 20000}

# Querying

In [ ]:
from pprint import pprint

def search_pinecone(query):
    # extract named entities from the query
    ne = extract_named_entities([query])[0]
    # create embeddings for the query
    xq = retriever.encode(query).tolist()
    # query the pinecone index while applying named entity filter
    xc = index.query(xq, top_k=10, include_metadata=True, filter={"named_entities": {"$in": ne}})
    # extract article titles from the search result
    r = [x["metadata"]["subject"] for x in xc["matches"]]
    return pprint({"Extracted Named Entities": ne, "Result": r})

Now try a query.

In [ ]:
query = "جاهای دیدنی تهران"
search_pinecone(query)

{'Extracted Named Entities': ['تهران'],
 'Result': ['کاخ گلستان',
            'از تهران یک روزه به کجا برویم',
            'عیار طبیعت تهران',
            'آشنایی با عمارت کاظمی',
            'نمایشگاه بین\u200cالمللی تهران کجاست ؟',
            'تفریحی جذاب و متفاوت در آبشار تهران',
            'بازار سنتی تجریش مکانی برای خرید',
            'قیمت آپارتمان در تهران',
            'مشاهدات پروفسور سایمون بل از طراحی شهری تهران',
            'دریاچه چیتگر را به خاطر بسپارید']}


In [ ]:
query = "جاهای دیدنی اصفهان"
search_pinecone(query)

{'Extracted Named Entities': ['اصفهان'],
 'Result': ['سوغات و صنایع دستی اصفهان',
            'سوغات و صنایع دستی اصفهان',
            'رستوران ترنج اصفهان',
            'سفر به زنجان، مقصد همیشه ارزان و در دسترس(۱)',
            'قضاوت نابجا',
            'مسیر طراحی سایت در اصفهان چگونه است؟',
            'مسیر طراحی سایت در اصفهان چگونه است؟',
            'طرز پخت قیمه ریزه',
            'خیابان\u200cهای طرح ترافیک در اصفهان در چه محدوده ای قرار دارند؟',
            'شروع داستان سئو سایت در اصفهان']}
